<a href="https://colab.research.google.com/github/luigiselmi/dl_tensorflow/blob/main/transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning for text: transformers
The transformer architecture is based on the attention mechanism. The embeddings are used to provide a basic semantic to the words of a vocabulary but most often the semantic of a word depends on the context, that is from the other words in a sentence. One way to weight the relevance of a word in a sentence is to compute the dot product of its vector representation and that of the other words. The attention mechanism was developed for machine translation where two sentences, a source and a target, are compared in a similar way as in a search engine where relevant documents are searched using a query that should return a ranked set of (key, value) pairs. The query, the keys and the values are kept separate as attention heads. The same mechanism can be used with only one sentence and in this case the mechanism is called self-attention. The attention mechanism is implemented in keras by the [MultiHeadAttention](https://keras.io/api/layers/attention_layers/multi_head_attention/) layer. The attention layer in combination with a residual connection layer, to avoid the vanishing gradients issue, and a normalization layer, specific to sequence data, are the building block of the [TransformerEncoder](https://keras.io/api/keras_nlp/modeling_layers/transformer_encoder/) layer. A transformer has two parts: an encoder and a decoder. The encoder can be used in text classification tasks as in the IMDB reviews classification.  

In [4]:
!pip install tensorflow==2.8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 51.0 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: go

In [5]:
import tensorflow as tf
import keras
from keras import layers
print('Tensorflow version: {}'.format(tf.__version__))
print('Keras version: {}'.format(keras.__version__))

Tensorflow version: 2.8.0
Keras version: 2.8.0


## Data preparation
We download the IMDB dataset

In [6]:
!wget -nv 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz' -P './data'

2024-10-31 13:37:33 URL:https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz [84125825/84125825] -> "./data/aclImdb_v1.tar.gz" [1]


In [7]:
!tar -xf data/aclImdb_v1.tar.gz -C data/

In [8]:
!rm -r data/aclImdb/train/unsup

In [9]:
import os, pathlib, shutil, random
base_dir = pathlib.Path("data/aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname, val_dir / category / fname)

In [10]:
from tensorflow import keras
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory('data/aclImdb/train', batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory('data/aclImdb/val', batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory('data/aclImdb/test', batch_size=batch_size)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [11]:
max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
  max_tokens=max_tokens,
  output_mode="int",
  output_sequence_length=max_length,
)

In [12]:
text_vectorization.adapt(text_only_train_ds)
int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [13]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

In [14]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [15]:
inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 transformer_encoder (Transf  (None, None, 256)        543776    
 ormerEncoder)                                                   
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257   

In [ ]:
callbacks = [
  keras.callbacks.ModelCheckpoint("transformer_encoder.keras",
  save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

Epoch 1/20
625/625 [==============================] - 2289s 4s/step - loss: 0.4820 - accuracy: 0.7811 - val_loss: 0.3630 - val_accuracy: 0.8382
Epoch 2/20
625/625 [==============================] - 2238s 4s/step - loss: 0.3118 - accuracy: 0.8647 - val_loss: 0.3431 - val_accuracy: 0.8532
Epoch 3/20
625/625 [==============================] - 2203s 4s/step - loss: 0.2440 - accuracy: 0.9004 - val_loss: 0.3144 - val_accuracy: 0.8722
Epoch 4/20
510/625 [=======================>......] - ETA: 6:12 - loss: 0.1907 - accuracy: 0.9254

In [3]:
#model = keras.models.load_model("transformer_encoder.keras", custom_objects={"TransformerEncoder": TransformerEncoder})
#print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

## Positional embedding
So far there is no information about the position of a token in a sequence, that is the position of a word in a sentence. The position of a word in a sentence helps in understanding the semantics of each word and of the full sentence. For example, in many western languages a name is followed by a verb and not the other way around. We could add the positional information by adding a dimension to the embedding with the index of the word in the sentence. This could be achieved by mapping an integer to a range of real values, e.g. [-1, 1]. A better approach is to let the transformer learn the position of a word in a sentence.

In [16]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [17]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

We replace the default embedding layer with the positional one

In [19]:
inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
callbacks = [
  keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",
  save_best_only=True)
]
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding_1 (Pos  (None, None, 256)        5273600   
 itionalEmbedding)                                               
                                                                 
 transformer_encoder_2 (Tran  (None, None, 256)        543776    
 sformerEncoder)                                                 
                                                                 
 global_max_pooling1d_2 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                           

In [ ]:
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

Epoch 1/20
625/625 [==============================] - 2063s 3s/step - loss: 0.4787 - accuracy: 0.7811 - val_loss: 0.3276 - val_accuracy: 0.8632
Epoch 2/20
625/625 [==============================] - 2034s 3s/step - loss: 0.2318 - accuracy: 0.9092 - val_loss: 0.3122 - val_accuracy: 0.8904
Epoch 3/20
625/625 [==============================] - 2042s 3s/step - loss: 0.1753 - accuracy: 0.9354 - val_loss: 0.3756 - val_accuracy: 0.8830
Epoch 4/20
625/625 [==============================] - 2042s 3s/step - loss: 0.1412 - accuracy: 0.9496 - val_loss: 0.3319 - val_accuracy: 0.8896
Epoch 5/20
625/625 [==============================] - 2059s 3s/step - loss: 0.1179 - accuracy: 0.9576 - val_loss: 0.4683 - val_accuracy: 0.8864
Epoch 6/20
 49/625 [=>............................] - ETA: 28:46 - loss: 0.1349 - accuracy: 0.9515

In [ ]:
model = keras.models.load_model("full_transformer_encoder.keras",
                          custom_objects={"TransformerEncoder": TransformerEncoder,
                          "PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")